# Walt Erbach's Calculator

Walter Erbach, a respected indoor model builder, was also  a Professor of Engineering Mechanics at the University of Nebraska who retired from teaching in 1984. He authored several papers on the theory of indoor model designs that were published in the NFFS Symposiums. At the time when he was most active, computers were huge machines that were only available in big organizations like universities. 

In the 1980 edition of the Symposium, Walt presented a program that used simple theory to calculate the power needed for an indoor model to maintain level flight. This program was originally written in *FORTRAN*, but was rewritten in *Basic* for the Commodore 64, one of the first machines available for home use. In this note, we will recreate code he wrote, this time using Python.

In [1]:
import numpy as np
import scipy

In [2]:
## Test Model Data

Walt decided to use the McBride B7 airfoil for his test model since he found lift and drag coefficient data in a *Frank Zaic Yearbook*.

In [3]:
C_l = [0.06, 0.135, 0.2, 0.25, 0.3, 0.35, 0.395, 0.44]
C_d = [0.008, 0.009, 0.01, 0.012, 0.014, 0.019, 0.024, 0.0335]
alpha = [-2.0, 0.0, 2.0, 4.0, 6.0, 8.0, 10.0, 12.0]

These three variables define *Python* lists.

The program also defines a few parameters for the model we are going to need.

In [4]:
SW = 150.0     # model wing area in square inches
WC = 5.5       # wing chord in inches
WI = 4         # wing incidence in degrees
WH = 3.0       # wing height in inches
W = 0.070      # model weight in ounces
PCW = 0.40     # ratio of stab area to wing area
TA = 17.0      # distance from 25% wing chord to 25% stab chord

The **TA** parameter is the distance from the *Mean Aerodynamic Chord (MAC)* of the wing to the *MAC* of the tail. 

## Using Curve Data

The lift and drag coefficient data are not very useful for our new code. What we need is a way to use that data to produce a function that can give us values for any point in the range of the data, and possibly a bit beyond both ends. This is a common problem in the *data science* research world, and there many *Python* tools available for dealing with this. The **scipy** library has a useful function that uses *spline interpolation* that we will use.

In [5]:
from scipy.interpolate import InterpolatedUnivariateSpline

def fit_curve(xp, yp):
    xi = np.array(xp)
    yi = np.array(yp)
    order = 1
    s = InterpolatedUnivariateSpline(xi, yi, k=order)
    return s

This function takes two lists of data points, and returns a function we can use to get values both inside and outside of the range of those points. The extrapolation will simply extend the curve found at the ends for some distance past those ends. Obviously, this should be used with caution.

*Python's* ability to return a function in a variable is very handy, and we will use that feature to generate functions that will let use determine both lift and drag coefficients for any input angle of attack.

In [6]:
CL = fit_curve(alpha, C_l)
CD = fit_curve(alpha, C_d)

Now, **CL** and **CD** are the names of functions we can use to get values for $C_l$ and $C_d$. Let's try them out:

In [7]:
angle = 6
cl_w = float(CL(angle))
cd_w = float(CD(angle))
print(cl_w, cd_w)

0.3 0.014


The model used in Walt's code looks pretty conventional. Below is a figure from his paper that shows all of the cryptic variables he used in his *BASIC* code. Back when this was written, using readable names was unthinkable since memory was expensive! Programmers kept things short! I will fix that in the final *Python* program I generate later.

![Erbach Model](./assets/images/erbach-model.png)

## basic Aerodynamic Definitions

Before we go further, lets review some of the basic definitions we will need in our program.

### Lift and Drag coefficients:

$$
C_l = \frac{2L}{\rho u^2 S}
$$

$$
C_d = \frac{2D}{\rho u^2 S}
$$

Where:

- **L** is the lift force
- **D** is the drag force
- **S** is a reference surface area
- **V** is the flight velocity
- $\rho$ is the air density.

### Dynamic Pressure

The *dynamic pressure* is related to the force a moving body of air can exert on a body.

$$
q = \frac{\rho u^2}{2}
$$

### Pitching Moment

I *Moment* is a torque, meaning force times length. The *pitching moment is a nondimensional number that is related to the moments produced by all the aerodynamic forces acting on the model. 

$$
C_m = \frac{M}{qSc}
$$

Where:

- **M** is the total moment (about the *CG*)
- **q** is the dynamic pressure
- **S** is a reference area
- **c** is the wing chord

We can rearrange the *lift coefficient* equation to find the flight velocity:

$$
V = \sqrt{\frac{2 L}{\rho S C_l}}
$$

Similarly, we can use the same equation to calculate the lift force:

$$
L = \frac{\rho S V^2 C_l}{2}
$$

But, using the definition of the *dynamic pressure**, this becomes:

$$
L = q S C_l
$$

and:

$$
D = q S C_d
$$

### Power

Power is simple a force (weight) times a velocity. We will use this fact later. 

These equations are the basis for the first part of Walt's code. But before we start work on that, we need to consider the units for all the terms we have introduced.

## Example Calculations

Before we work through our equations, let's just use the numbers from Walt's code and his expressions to see what we get.

### Basic Test Model Data

Now we can define a few dimensional model values for our work. Most of these variable names come from Walt's code:

In [8]:
W = 0.070        # model weight in ounces
WT = W / 16.0    # model weight in pounds
AW = 150.0       # wing surface area in square inches
SW = AW/144.0
PCW = 0.4        # stab size as a percentage of wing area
WH = 3.0         # wing height in inches
RHO2 = 0.00119   # shown as a bare number in the code 

Now, we will calculate the velocity, **VE**:

In [9]:
alpha_wing = 2
alpha_stab = -2
VE = (WT/(RHO2*SW*(CL(alpha_wing)+PCW*CL(alpha_stab))))**0.5
VE

3.9694209301872236

This matches the value generated from Walt's code, as shown by *Figure 1* from his paper!

### Dimensional Analysis

Just plugging in numbers to equations might generate results, but you do not know if they represent anything real. We need to be sure that all of the variables used in our calculations use the right units! Walt defined many variables in his program, and added comments to show the units he was using. The *Python* **pint** library makes it easy to add dimensional units to our variables and make sure our calculated values are correct!

To demonstrate this, we need to initialize **pint** and create a *Unit Registry* gadget we can use to attach units to variables:

In [10]:
import pint
u = pint.UnitRegistry()

Now, **u** is a powerful thing that knows a huge amount about units! Let's try it out on the surface area:

In [11]:
# S_w is the surface area of the wing
S_w = 150.0 * u.inches ** 2
S_w.to('meters**2')

0.096774 <Unit('meter ** 2')>

Now that is pretty cool! I defined the area in square inches and asked for the value in square meters. All without doing anything to figure out that conversion. 

Internally, **pint** converts everything to *Standard International* (metric) base units. We can convert back to whatever unit system we like, as we shall see in the examples below.

There are conversion factors in this code, but we have no way to know everything is correct. Let's use **pint** to attach units to these numbers and see what happens.

In [12]:
WT = 0.070 * u.ounces
AW = 150.0 * u.inch**2
PCW = 0.4
SS = PCW * AW
WH = 3.0 * u.inch 

Density is the mass of something per unit volume. For air, we use the symbol $\rho$ for this quantity. The "magic number", which I called **RHO2** above, in Walt's code which looks to be ${\rho/2}$. I checked this by looking up the air density in the *Standard Atmosphere* model used by researchers. You can find this at [Standard Atmosphere Calculator](https://www.digitaldutch.com/atmoscalc/) Therefor, the correct value for density is:

In [13]:
density = 0.00119 * 2 * u.slugs/u.ft**3 # air density from program
density.to_base_units()

1.2266015877758076 <Unit('kilogram / meter ** 3')>

I am not sure where Walt got this number, since it matches Lincoln, NE when it is about 6 below zero!

#### Mass vs Weight

One of the biggest headaches in generating proper code here is making sure we distinguish between the mass of something and the weight of something.

From *Newton's Second Law*, we know that;

$$ 
F = m a
$$

Where **F** is the force (weight) of an object, **m** is the *mass* of that object, and **g** is the acceleration due to gravity. To get the mass of something, we divide its weight by **g**.

**pint** makes it easy to  see the weight of the model using in *SI* units:

In [14]:
WT.to_base_units()

0.0019844666187500007 <Unit('kilogram')>

In [15]:
(WT/(u.gravity*u.gravity*AW)).to_base_units()

0.0002132277648529057 <Unit('kilogram * second ** 4 / meter ** 4')>

### Flight Velocity

Now we can recreate the velocity calculation with proper units attached:

In [16]:
k1 = (WT * 2 * u.gravity) / (density * AW)
k2 = (CL(alpha_wing)+PCW*CL(alpha_stab))
VE = (k1/k2)**0.5
VE.to('ft/sec')

3.9694209301872245 <Unit('foot / second')>

Again, this matches the value from Walt's paper.

### Component Lift and Drag

Now that we know the flight velocity, we can calculate the lift and drag generated by both the wing and stab. We will use Walt's variable names to match the model figure.

First, lets calculate the *dynamic pressure* $(\rho u^2)/2$ for the flight velocity:

In [17]:
DP = density /2 * VE **2 
DP.to_base_units()

0.8977548558812976 <Unit('kilogram / meter / second ** 2')>

Pressure acting on an area produces a force. This will be used later.

In [18]:
AWL = AW * DP * CL(alpha_wing)
AWL.to_base_units()

0.017375865684611334 <Unit('kilogram * meter / second ** 2')>

In [19]:
CTL = PCW * AW * DP * CL(alpha_stab)
CTL.to_base_units()

0.00208510388215336 <Unit('kilogram * meter / second ** 2')>

In [20]:
Lift = (AWL + CTL)
Lift.to_base_units()

0.019460969566764696 <Unit('kilogram * meter / second ** 2')>

The units here are not those for a force (weight) so we need to divide by the acceleration of gravity:

In [21]:
Lift_d = Lift / u.gravity
Lift_d.to_base_units()

0.0019844666187500007 <Unit('kilogram')>

In [22]:
WT.to('oz')

0.07 <Unit('ounce')>

Now, our lift matches the weight of the model. We have equations we can use!

Next, let's calculate the drag force

In [23]:
BWD = AW * DP * CD(alpha_wing)
DTD = PCW * AW * DP * CD(alpha_stab)
Drag = (BWD + DTD)/u.gravity
Drag.to('oz')

0.004125000000000001 <Unit('ounce')>

### Level Flight Power

The power required to maintain level flight must equal the total drag. From Erbach's paper, that power should be this:

In [24]:
power = 0.196 * u.inch * u.oz / u.sec
power.to_base_units()

0.00014113526592550004 <Unit('kilogram * meter / second')>

Dividing this by the flight velocity should give us the drag force., the drag force shound be:

In [25]:
drag_e = (power/VE)
drag_e.to('oz')

0.004114789945586079 <Unit('ounce')>

Well, we have a slight problem here. The units match, but the numbers are not agreeing all that well. I suspect the problem has to do with the poor precision when working on the Commodore 64. The "64" has to do with available memory: 64K!, so high accuracy was not going to be available! We will need to return to this issue later.

### Pitching Moment

We have the lift and drag values available, all we need to do this calculation is to figure out the moment arms (distances from the CG) for each of those forces)

Erbach's code uses these formulas for the distances defined in the model diagram above. A positive moment causes a nose up condition by convention.

In [26]:
import math
CG = 0.5
WC = 5.5 * u.inch
TA = 17.0 * u.inch
DW = WC * (CG - 0.25)
EWLA = WH * math.sin(alpha_stab) - DW * math.cos(alpha_stab)
FWDA = WH * math.cos(alpha_stab) - DW * math.sin(alpha_stab)
GTLA = (TA - DW)*math.cos(alpha_stab)
HTDA = (TA - DW)* math.sin(alpha_stab)

JMWL = -AWL * EWLA
KWMD = +BWD * FWDA
LMTL = -CTL * GTLA
MWTD = -DTD * HTDA

M = (JMWL + KWMD + LMTL + MWTD)/u.gravity
M.to('in oz')

0.19771138925432696 <Unit('inch * ounce')>

This is close to the value Walt published, but again is not exact.

## Parametric Study

The rest of the code in Walt's program set up a parametric study that produce "terrifying yards of tabulated data". Unfortunately, Walt had to manually convert this data into plots for his report. We are much too lazy for that sort of thing, so we will let *Python* and **matplotlib** do that for us.

For this section, we will recreate *Figure 2* and *Figure 3* from Walt's report. 

To generate this plot, we need to write a few functions that we can use for a parametric study.

Fortunately, **numpy** will be a big help here!

In [27]:
def VE_alpha(wa, sa):
    """Given an list of alpha values for wing and stab, return a list of velocities"""
    k1 = (WT * 2 * u.gravity) / (density * AW)
    k2 = (CL(wa)+PCW*CL(sa))
    VE = (k1/k2)**0.5
    return VE

lets set up a test case:

In [28]:
stab_angles = np.linspace(-2,12,8)
stab_angles

array([-2.,  0.,  2.,  4.,  6.,  8., 10., 12.])

In [29]:
wing_angles = stab_angles + 4

In [30]:
wing_angles

array([ 2.,  4.,  6.,  8., 10., 12., 14., 16.])

Isn't **numpy** neat?

In [31]:
VEset = VE_alpha(wing_angles, stab_angles)
VEset.to_base_units()

Magnitude,[1.209879499521066 1.0385546040698463 0.9289114772182224 0.853610739223475 0.797925809083669 0.7518862285003758 0.7141026433978803 0.6814967305138373]
Units,meter/second


In [32]:
VEset.to('ft/sec')

Magnitude,[3.9694209301872245 3.4073313781819103 3.0476098333931185 2.80056016805602 2.617866827702326 2.4668183349749864 2.342856441594096 2.235881661790805]
Units,foot/second


Next we generate functions that will calculate the lift and drag sets from the velocity sets.

In [35]:
def lift_alpha(V, wa, sa):
    DP = density / u.gravity /2 * V **2
    AWL = AW * DP * CL(wa)
    CTL = SS * DP * CL(sa)
    return (AWL + CTL)

In [36]:
lift_set = lift_alpha(VEset, wing_angles, stab_angles)
lift_set.to_base_units()

Magnitude,[0.0019844666187500007 0.0019844666187500003 0.0019844666187500007 0.0019844666187500007 0.0019844666187500007 0.0019844666187500007 0.0019844666187500007 0.0019844666187500003]
Units,kilogram


In [43]:
def drag_alpha(V, wa, sa):
    DP = density / u.gravity /2 * V **2
    BWD = AW * DP * CD(wa)
    DTD = SS * DP * CD(sa)
    return (BWD + DTD)

In [44]:
drag_set = drag_alpha(VEset, wing_angles, stab_angles)
drag_set

Magnitude,[0.004125000000000001 0.0035921052631578943 0.0033157894736842116 0.0037022222222222226 0.004023300970873787 0.004960344827586207 0.00572628304821151 0.00653399433427762]
Units,ounce


With velocities created, we can build a set of power values

In [48]:
def P_alpha(v,d, wa, sa):
    """Given an list of alpha values for wing and stab, return a list of velocities"""
    P = v*d/u.gravity   
    return P

In [49]:
Pset = P_alpha(VEset, drag_set, wing_angles, stab_angles)
Pset.to_base_units()

Magnitude,[1.4427502333295317e-05 1.078458586205112e-05 8.90402477417704e-06 9.135817931712513e-06 9.280473116250776e-06 1.078174561983287e-05 1.1821117502952309e-05 1.287263966544138e-05]
Units,kilogram second


In [38]:
def CMset(wa,sa, dp):
    a = AW * dp * CL(wa)
    b = SS * dp * CD(sa)
    c = AW * dp * CL(wa)
    d = SS * dp * CD(sa)
    CG = 0.5
    WC = 5.5 * u.inch
    TA = 17.0 * u.inch
    DW = WC * (CG - 0.25)
    EWLA = WH * np.sin(sa) + DW * np.cos(sa)
    FWDA = WH * np.cos(sa) - DW * np.sin(sa)
    GTLA = (TA - DW)*np.cos(sa)
    HTDA = (TA - DW)* np.sin(sa)

    JMWL = -a * EWLA
    KWMD = +b * FWDA
    LMTL = -c * GTLA
    MWTD = -d * HTDA

    return (JMWL + KWMD + LMTL + MWTD)/u.gravity

In [39]:
DPset = density /2 * VEset **2
DPset.to_base_units()
CMvals = CMset(wing_angles, stab_angles, DPset) 

In [40]:
CMvals

Magnitude,[0.6268588971075494 -0.9761315789473685 0.2278970096392686 0.736736743507101 -0.8255521214122433 -0.042091135038329015 0.8463483858767323 -0.653778086698054]
Units,inch ounce
